In [2]:
# | results: hide
import pandas as pd
from datetime import datetime
import requests
import holoviews as hv
import locale

locale.setlocale(locale.LC_TIME, "de_DE.UTF-8")
hv.extension("bokeh", logo=False)

# Dynamischer Stromtarif vs. Fixpreistarif in Kombination mit Photovoltaikanlage

In unserer Eigentümergemeinschaft kann ich den Stromverbrauch der vier Wohneinheiten [digital in Echtzeit auslesen](https://github.com/grst/energymeter). Bereits im Vergangenen Jahr habe ich anhand der Verbrauchsdaten von 2024 unseren momentanen Stromtarif mit [Tado Hourly](https://energy.tado.com/) verglichen. Dabei kam ich zu dem Ergebnis, dass der dynamische Stromtarif auch ohne steuerbare Verbraucher  für alle Wohneinheiten (geringfügig) günstiger ist. 

In diesem Post gibt es ein Update mit den Daten von 2025. Zusätzlich berücksichtige ich in diesem Jahr den Strom, der von der Photovolatik (PV)-anlage produziert wurde. Möglicherweise ist in Kombination mit PV der dynamische Stromtarif gar nicht mehr günstiger, weil der Strom vermutlich meist dann im Überfluss vorhanden ist, wenn man selber gerade Strom vom eigenen Dach bezieht. 

Die Rohdaten und das Python Notebook mit der Analyse stehen [auf GitHub zur Verfügung](https://github.com/grst/dynamischer-stromtarif). 

## Die Tarife

Es gibt eine Unzahl an unterschiedlichen Stromtarifen in Deutschland mit teilweise sehr unterschiedlichen
Preisstrukturen. Hier betrachte ich nur zwei Tarife: 

 * [AÜW Allgäustrom Basis](https://auew.de/privatkunden/strom/allgaeustrom-basis/), unser bisheriger Anbieter, Preisniveau für 2025
 * [Tado Hourly][]

Die Tarife gestalten sich wie folgt (alle Preise inkl. MwSt):

| Tarif | Arbeitspreis pro kWh | Grundpreis pro Monat |
| -- | -- | -- |
| AÜW | 34,62 ct (Strompreis + Steuern + Netzgebühr) | 15,68 EUR |
| tado | Epex Spot Day Ahead <br> + 19% MwSt <br> + 1,785ct (Aufschlag) <br> + 21,540 ct (Netzgebühr + Steuern) | 16,60 EUR |

: Verglichene Stromtarife {.striped .hover tbl-colwidths="[25,50,25]"}


Die Grundpreise sind sehr ähnlich, daher betrachte ich nur die Arbeitspreise. 

[Tado Hourly]: https://energy.tado.com/
[Tibber]: https://tibber.com/de

## Die Wohneinheiten

| Wohnung | Beschreibung |
| -- | -- |
|Wohnung 1 | Ganzjährig bewohnt von einer alleinstehenden Person. Ca. 25 Jahre alte Haushaltsgeräte. Aus historischen Gründen hängen diverse gemeinschaftliche Verbraucher (Treppenhaus, Heizung, ...) zusätzlich auf diesen Zähler |
|Wohnung 2 | Ganzjährig bewohnt von zwei Personen. Teilweise erneuerte Haushaltsgeräte. Eigene 3kWp Photovolatikanlage mit 6kWh Speicher hinter dem Zähler. |
|Wohnung 3 | Ganzjährig bewohnt von einer Familie mit zwei kleinen Kindern. Neubauwohnung mit modernen Haushaltsgeräten. |
|Wohnung 4 | Sporadisch genutzte Ferienwohnung |

: Verbrauchsprofile vier Wohnungen {.striped .hover tbl-colwidths="[25,75]"}

## Die PV Analage

Wir haben eine 24 kWp PV Analge mit 22 kWh Stromspeicher, die von allen Wohneinheiten gemeinsam genutzt wird. Anlage und Speicher sind so dimensioniert dass wir an einem sonnigen, schneefreien Dezembertag zu 100% autark sind. Insgesamt erreichten wir einen im Jahr 2025 einen Autarkiegrad von 83%. 

In [3]:
VAT = 1.19
YEAR = 2025
TADO_FIXED_FEE_PER_KWH = 0.21540  # EUR/kWh
AEUW_FEE_PER_KWH = 0.3462  # EUR/kWh
BASE_FEE_AUEW = 15.68

In [4]:
# Get hourly prices from awattar API
begin = datetime.fromisoformat("2020-01-01").timestamp()
end = datetime.now().timestamp()
# API endpoint URL
url = f"https://api.awattar.de/v1/marketdata?start={int(begin) * 1000}&end={int(end) * 1000}"

# Fetch JSON data from the API
response = requests.get(url)
data = response.json()

# Convert JSON data to Pandas DataFrame
hourly_price = pd.DataFrame(data["data"])

hourly_price["start"] = pd.to_datetime(hourly_price["start_timestamp"], unit="ms")
hourly_price["end"] = pd.to_datetime(hourly_price["end_timestamp"], unit="ms")
hourly_price["marketprice"], hourly_price["unit"] = hourly_price["marketprice"] / 1000, "EUR/kWh"
hourly_price["start_day"] = hourly_price["start"].dt.strftime(
    "%m-%d %H:%M:%S"
)  # day without year (for inter-year comparison)
hourly_price["real_price"] = hourly_price["marketprice"] * VAT + TADO_FIXED_FEE_PER_KWH

In [5]:
# Get recorded interval consumption and production data
all_interval = pd.read_csv("../data/2025_messwerte_pseudonymisiert_3600s_interval.csv", parse_dates=["time"])

In [6]:
consumption = all_interval.loc[lambda x: x["name"].isin(["Wohnung 1", "Wohnung 2", "Wohnung 3", "Wohnung 4"])]
production = all_interval.loc[
    lambda x: x["name"].isin(
        ["Sunny_Island_Batterie_entladen", "Sunny_Island_Netzbezug", "Sunny_Tripower_Gesamtertrag"]
    )
]

In [7]:
# Calcuate fraction of production by source (PV, Battery, Grid) for each hour
production_relative = (
    production.groupby("time")
    .apply(lambda x: x.assign(frac=x["Wh"] / x["Wh"].sum()))
    .reset_index(drop=True)
    .drop(columns=["Wh"])
    .rename(columns={"name": "source"})
)

/tmp/ipykernel_6179/1514857436.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.assign(frac=x["Wh"] / x["Wh"].sum()))


In [ ]:
# Calculate cost per hour, considering only the fraction that was drawn from the grid
cost_df = (
    consumption.merge(production_relative, on="time")
    .merge(hourly_price, left_on="time", right_on="start")
    .loc[lambda x: x["source"] == "Sunny_Island_Netzbezug"]
    .assign(Wh_grid=lambda x: x["Wh"] * x["frac"])
    .assign(
        # only energy from grid
        price_tado=lambda x: x["Wh_grid"] * x["real_price"] / 1000,
        price_auew=lambda x: x["Wh_grid"] * AEUW_FEE_PER_KWH / 1000,
        # all energy, theoretically, for comparison
        price_tado_no_pv=lambda x: x["Wh"] * x["real_price"] / 1000,
        price_auew_no_pv=lambda x: x["Wh"] * AEUW_FEE_PER_KWH / 1000,
    )
)

In [ ]:
cost_per_month = (
    cost_df.groupby("name")
    .apply(
        lambda x: x.set_index("time")[
            ["Wh", "Wh_grid", "price_tado", "price_auew", "price_tado_no_pv", "price_auew_no_pv"]
        ]
        .resample("MS")
        .sum()
    )
    .reset_index()
    .assign(month=lambda x: pd.to_datetime(x["time"]).dt.strftime("%b"))
    .assign(month=lambda x: pd.Categorical(x["month"], categories=x["month"].unique()))
)

/tmp/ipykernel_6179/3352185342.py:3: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(


In [ ]:
cost_per_year = cost_df.groupby("name")[
    ["Wh", "Wh_grid", "price_tado", "price_auew", "price_tado_no_pv", "price_auew_no_pv"]
].sum()

## Stromkosten 2025 -- ohne PV

Was hätten die jeweiligen Wohneinheiten 2025 bezahlt, wenn keine PV Anlage installierte wäre?

In [45]:
cost_per_year.reset_index().rename(
    columns={"name": "Wohneinheit", "price_tado_no_pv": "Tado (EUR)", "price_auew_no_pv": "AÜW (EUR)"}
).assign(
    **{
        "Verbrauch (kWh)": lambda x: x["Wh"] / 1000,
        "Ersparnis (EUR)": lambda x: x["AÜW (EUR)"] - x["Tado (EUR)"],
        "Ersparnis (%)": lambda x: (1 - x["Tado (EUR)"] / x["AÜW (EUR)"]) * 100,
    }
).set_index("Wohneinheit")[["Verbrauch (kWh)", "Tado (EUR)", "AÜW (EUR)", "Ersparnis (EUR)", "Ersparnis (%)"]].style.format(
    "{:.2f}"
)

,Verbrauch (kWh),Tado (EUR),AÜW (EUR),Ersparnis (EUR),Ersparnis (%)
Wohneinheit,,,,,
Wohnung 1,3637.10,1173.75,1259.16,85.41,6.78
Wohnung 2,1730.64,587.96,599.15,11.19,1.87
Wohnung 3,1973.20,651.05,683.12,32.08,4.70
Wohnung 4,583.84,189.53,202.13,12.59,6.23


Die folgende Grafik zeigt den Vergleich monatsweise:

In [ ]:
hv.Table(
    cost_per_month.rename(
        columns={"name": "Wohneinheit", "price_tado_no_pv": "Tado", "price_auew_no_pv": "AÜW", "month": "Monat"}
    )[["Wohneinheit", "Monat", "Tado", "AÜW"]].melt(
        ["Wohneinheit", "Monat"], var_name="Tarif", value_name="Kosten (EUR)"
    )
).to.bars(["Monat", "Tarif"], "Kosten (EUR)").opts(width=450, xrotation=90, legend_position="right")

:HoloMap   [Wohneinheit]
   :Bars   [Monat,Tarif]   (Kosten (EUR))

In [ ]:
hv.Table(cost_per_month)

[Wie bereits 2024](https://grst.github.io/dynamischer-stromtarif/dynamischer-stromtarif.html#vergleich-f%C3%BCr-2024) wären die Kosten des dynamischen Stromtarifs geringer als mit dem Fixpreistarif. Allerdings sind die Einsparungen mit 1,9 - 6,8% geringer als im Vorjahr (10-13%). Das könnte unter anderem daran liegen, dass der fixe Anteil pro kWh (Netzgebühren + Steuern) 2025 mit 21,54 ct/kWh nochmals deutlich höher liegt als im Vorjahr (19,116 ct/kWh). 

## Stromkosten 2025 -- mit PV

Nun derselbe Vergleich, allerdings unter Berücksichtigung der PV-Anlage. D.h. bei der Berechnung wurde für jedes 
Stundenintervall nur der Anteil berücksichtigt, der aus dem Stromnetz bezogen wurde. 

In [43]:
cost_per_year.reset_index().rename(
    columns={"name": "Wohneinheit", "price_tado": "Tado (EUR)", "price_auew": "AÜW (EUR)"}
).assign(
    **{
        "Verbrauch (kWh)": lambda x: x["Wh"] / 1000,
        "davon Netzbezug (%)": lambda x: x["Wh_grid"] / x["Wh"] * 100,
        "Ersparnis (EUR)": lambda x: x["AÜW (EUR)"] - x["Tado (EUR)"],
        "Ersparnis (%)": lambda x: (1 - x["Tado (EUR)"] / x["AÜW (EUR)"]) * 100,
    }
).set_index("Wohneinheit")[["Verbrauch (kWh)", "davon Netzbezug (%)", "Tado (EUR)", "AÜW (EUR)", "Ersparnis (EUR)", "Ersparnis (%)"]].style.format(
    "{:.2f}"
)

,Verbrauch (kWh),davon Netzbezug (%),Tado (EUR),AÜW (EUR),Ersparnis (EUR),Ersparnis (%)
Wohneinheit,,,,,,
Wohnung 1,3637.10,16.41,208.91,206.64,-2.27,-1.10
Wohnung 2,1730.64,32.26,195.96,193.27,-2.69,-1.39
Wohnung 3,1973.20,16.60,120.35,113.40,-6.95,-6.12
Wohnung 4,583.84,13.51,28.82,27.31,-1.51,-5.53


Die folgende Grafik zeigt den Vergleich monatsweise: 

In [46]:
hv.Table(
    cost_per_month.rename(
        columns={"name": "Wohneinheit", "price_tado": "Tado", "price_auew": "AÜW", "month": "Monat"}
    )[["Wohneinheit", "Monat", "Tado", "AÜW"]].melt(
        ["Wohneinheit", "Monat"], var_name="Tarif", value_name="Kosten (EUR)"
    )
).to.bars(["Monat", "Tarif"], "Kosten (EUR)").opts(width=450, xrotation=90, legend_position="right")

:HoloMap   [Wohneinheit]
   :Bars   [Monat,Tarif]   (Kosten (EUR))

In Kombination mit PV Anlage ist der dynamische Stromtarif aufs Jahr 2025 gerechnet also um wenige Euro *teurer* als unser
momentaner Fixpreistarif. Dies liegt daran, dass mit PV-Anlage und Stromspeicher hauptsächlich Strom in den Wintermonaten
bezogen wird, in denen der durchschnittliche Börsenpreis höher ist. 

## Fazit

Ohne steuerbare Großverbraucher (wie Wärmepumpe oder E-Auto) ist für einen normalen Haushalt der finanzielle Anreiz
auf einen dynamischen Stromtarif zu wechseln gering (Ersparnis von wenigen Euro pro Monat). Mit PV Anlage und Stromspeicher ist der Anreiz nicht vorhanden (Mehrkosten von wenigen Euro pro Jahr). 

Interessant wäre in Kombination mit einem dynamischen Stromtarif, den Stromspeicher im Winter mit Netzstrom zu laden wenn 
er gerade günstig ist (z.B. nachts). Das Einsparpotenzial bedarf einer separaten Analyse. 
